# Prepare raster files by clipped and aligned to a master for Deep Learning, every weekend.
This notebook demonstrates how you can use Buteo together with Rocketry to schedule the processing of geospatial data. </br> The example does the following:
1. Reads the raster content of a folder.
2. Clips all the rasters to a target in memory (Can be vector, raster).
3. Aligns all the resulting rasters in memory.
4. Combines all the rasters into a single numpy array and saves it to disc as "latest.npz".
5. Conducts all of the above step every Saturday afternoon after 16:00.

In [ ]:
!pip install rocketry
!pip install buteo

In [ ]:
import buteo as beo
import numpy as np
from rocketry import Rocketry

In [ ]:
ORIGIN_FOLDER = "/content/coarse_data/"
DESTINATION_FOLDER = "/content/cleaned_data/"
BOUNDS_FILE = "content/cleaned_data/bounds.tif" # can also be a vector.

In [ ]:
def ready_files():
    # Clip all the .tif files in the folder to the bounding box of a raster. (Works regardless of projections.)
    clipped = beo.clip_raster(ORIGIN_FOLDER + "*.tif:glob", BOUNDS_FILE, suffix="_clipped")

    # Aligns and reprojections all the rasters to a master file.
    aligned = beo.align_rasters(clipped, master=BOUNDS_FILE, suffix="_aligned")

    # Reads the rasters as a NumPy array and stacks them. (Only works if they are aligned.)
    stacked = beo.raster_to_array(aligned)

    # Save as a compressed NumPy array.
    np.savez_compressed(DESTINATION_FOLDER + "latest.npz", data=stacked)

    return True

In [ ]:
# Start the rocketry scheduling service and run it on Saturday afternoons.
app = Rocketry()

@app.task(rocketry.conds.weekly.on("Sat") & rocketry.conds.time_of_day.after("16:00"))
def do_daily():
    ready_files()